In [17]:
import numpy as np
from scipy.stats import norm

def run_sdt(n_trials=200, d_prime=1.5, std=1, criterion_c=0, epsilon=0.0001):
    mean_nontarget = 0
    mean_target = mean_nontarget + d_prime * std

    # criterion 설정: conservative or liberal
    criterion = mean_nontarget + criterion_c * std

    responses = []
    labels = []

    for _ in range(n_trials):
        if np.random.rand() < 0.5:
            sensory_input = np.random.normal(mean_target, std)
            labels.append(1)
        else:
            sensory_input = np.random.normal(mean_nontarget, std)
            labels.append(0)

        if sensory_input >= criterion:
            responses.append(1)
        else:
            responses.append(0)

    # Hit / False Alarm
    hits = 0
    false_alarms = 0

    for i in range(len(responses)):
        r = responses[i]
        l = labels[i]

        if r == 1 and l == 1:
            hits += 1

        if r == 1 and l == 0:
            false_alarms += 1

    n_targets = sum(labels)
    n_nontargets = len(labels) - n_targets

    # rate 계산
    if n_targets > 0:
        hit_rate = hits / n_targets
    else:
        hit_rate = epsilon

    if n_nontargets > 0:
        false_alarm_rate = false_alarms / n_nontargets
    else:
        false_alarm_rate = epsilon

    # 확률 보정
    hit_rate = min(max(hit_rate, epsilon), 1 - epsilon)
    false_alarm_rate = min(max(false_alarm_rate, epsilon), 1 - epsilon)

    # d', c 계산
    z_hit = norm.ppf(hit_rate)
    z_fa = norm.ppf(false_alarm_rate)
    d_empirical = z_hit - z_fa
    c_empirical = -0.5 * (z_hit + z_fa)

    print("[criterion c = ", criterion_c, "]")
    print("추정된 d':", round(d_empirical, 2))
    print("추정된 c:", round(c_empirical, 2), "\n")


    return d_empirical, c_empirical


In [23]:
run_sdt(criterion_c=1)
run_sdt(criterion_c=-1)

[criterion c =  1 ]
추정된 d': 1.44
추정된 c: 0.3 

[criterion c =  -1 ]
추정된 d': 2.75
추정된 c: -2.34 



(2.751594919354008, -2.343219025778705)